## Static Structure Factor

the `freud.diffraction` module provides the tools to calculate static structure factor S(q) [link to wiki?] which can be used to characterise structure of crystals, liquids or amorphous phases. In this notebook we will show how to calculate static structure factor of Lennard Jones liquid obtained from basic HOOMDv3 example [link to hoomd md]. We shall use both the "direct" method and the Debye scattering formula. The "direct" method calculates the static structure factor by constructing a k-space grid and applying the formula for S(q) to update the histograms on the k-space grid. The histograms are then integrated to give q-scalar dependence. Debye scattering formula on the other hand integrates out the orientations prior to histogram binning and thus provides a much faster algorithm giving worse results at low-k values then the "direct" method.

## freud.diffraction.StaticStructureFactorDebye

The `freud.diffraction.StaticStructureFactorDebye` class computes a one-dimensional [static structure factor](https://en.wikipedia.org/wiki/Structure_factor) for a set of $N$ points, based on the Debye scattering equation:

$$ S(\mathbf{q}) = {\frac{1}{N}} \sum_{j=1}^{N}\sum_{k=1}^{N}{\frac{\sin(qr_{jk})}{qr_{jk}}}$$

The points used here were generated from a HOOMD-blue simulation of a 1000-particle system subject to the Lennard-Jones potential. For a tutorial of how to run such a simulation, see 00-Introducing-HOOMD-blue in Glotzerlab's [hoomd-examples repository](https://github.com/glotzerlab/hoomd-examples).

## freud.diffraction.StaticStructureFactorDirect

The `freud.diffraction.StaticStructureFactorDirect` class computes a one-dimensional [static structure factor](https://en.wikipedia.org/wiki/Structure_factor) for a set of N points:

$$S(\mathbf{q}) = {\frac{1}{N}}\sum_{j=1}^{N}\sum_{k=1}^{N}\mathrm{e}^{-i\mathbf{q}\cdot(\mathbf{R}_{j} - \mathbf{R}_{k})}$$

The points used here were generated from a HOOMD-blue simulation of a 1000-particle system subject to the Lennard-Jones potential. For a tutorial of how to run such a simulation, see 00-Introducing-HOOMD-blue in Glotzerlab's [hoomd-examples repository](https://github.com/glotzerlab/hoomd-examples).

In [ ]:
import freud

# Read in the simulation data
import gsd.hoomd

traj = gsd.hoomd.open("../data/LJsampletraj.gsd", "rb")
import freud
import matplotlib.pyplot as plt

sfdirect = freud.diffraction.StaticStructureFactorDirect(bins=200, k_max=25, k_min=1)

sfdebye = freud.diffraction.StaticStructureFactorDebye(bins=200, k_max=25, k_min=1)

sfdebyeNoPBC = freud.diffraction.StaticStructureFactorDebye(bins=200, k_max=25, k_min=1)

# Accumulate S(q) over the entire trajectory
for frame in traj:
    box = frame.configuration.box
    points = frame.particles.position
    sfdirect.compute((box, points), reset=False)
    sfdebye.compute((box, points), reset=False)
    NoPBCbox = box
    NoPBCbox.PBC = False
    sfdebyeNoPBC.compute((NoPBCbox, points), reset=False)

# create legend stuff
plt.plot(sfdirect.bin_centers, sfdirect.S_k, label="direct")
plt.plot(sfdebyeNoPBC.bin_centers, sfdebyeNoPBC.S_k, label="debye no PBC")
plt.plot(sfdebye.bin_centers, sfdebye.S_k, label="debye with PBC")
plt.legend()
plt.show()

Both `freud.diffraction.StaticStructureFactorDirect` and `freud.diffraction.StaticStructureFactorDebye` classes support calculation of partial structure factors according to Faber-Ziman decomposition [link to docs??]. In this example we randomly split the system in two parts, one containing a third of the system and the other one containing the other two thirds of the system. The calculation of the partials requires the usage of nquery and N_tot to be sucessfull:

# freud.diffraction.StaticStructureFactorDirect

The `freud.diffraction.StaticStructureFactorDirect` class computes a one-dimensional [static structure factor](https://en.wikipedia.org/wiki/Structure_factor) $S(k)$ of a set of points. For an excellent introduction to the theory of scattering and $S(k)$, please refer to the documentation of the [dynasor package](https://dynasor.materialsmodeling.org/), which performs a number of calculations related to scattering. We have re-implemented the core method of static structure factor calculation from dynasor in **freud** with some additional performance optimizations in parallelized C++ code, as well as an interface to compute $S(k)$ that aligns with the APIs and conventions of the **freud** analysis library.

In [ ]:
L = 10
N = 10000
max_k_points = 200000
box, points = freud.data.make_random_system(L, N)
system = freud.AABBQuery.from_system((box, points))
A_points = system.points[: N // 3]
B_points = system.points[N // 3 :]
sf = freud.diffraction.StaticStructureFactorDirect(
    bins=100, k_max=100, k_min=1, max_k_points=max_k_points
)
sf.compute((system.box, B_points), query_points=A_points, N_total=N)
sf.plot()
plt.show()